In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.callbacks import EarlyStopping,TensorBoard
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer,scale
import pickle
import numpy as np
from keras import backend as K
import time
from os import path

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
#print(train_df.head())
train_X = train_df.drop(columns=['Severity'])
train_y = train_df[['Severity']]

test_X = test_df.drop(columns=['Severity'])
test_y = test_df[['Severity']]

if path.exists('le.txt'):
    le=pickle.load(open('le.pickle','rb'))
else:
    le = LabelBinarizer()
    le.fit(train_y)
    pickle.dump(le,open("le.pickle","wb"))

train_X=scale(train_X)
train_y = le.transform(train_y)
test_X=scale(test_X)
test_y = le.transform(test_y)

In [3]:
dense_layers = [3,5,7,8]
layer_sizes = [128]
#get number of columns in training data
n_cols = train_X.shape[1]
scores=[]
for dense_layer in dense_layers:
    for layer_size in layer_sizes:   
         NAME = "{}-nodes-{}-dense-{}".format(layer_size, dense_layer, int(time.time()))
         
         
         model = Sequential()

         #add layers to model

         model.add(Dense(250, activation='relu', input_shape=(n_cols,)))

         for i in range(dense_layer):
            model.add(Dense(layer_size, activation='relu'))
            model.add(Dropout(0.3))
        
         model.add(Dense(4, activation='sigmoid'))

         #compile model using accuracy to measure model performance
         tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
         model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

         #set early stopping monitor so the model stops training when it won't improve anymore
         early_stopping_monitor = EarlyStopping(patience=3)

         model.fit(train_X, train_y, validation_split=0.2, epochs=500, callbacks=[tensorboard,early_stopping_monitor])

         print(NAME)
         pickle.dump(model,open(NAME+'.pickle',"wb"))
         print('SCORE:',model.evaluate(test_X, test_y, verbose=0))
         scores.append(model.evaluate(test_X, test_y, verbose=0))
        

W0407 17:31:55.464111 12908 deprecation.py:506] From c:\program files\python37\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0407 17:31:56.373162 12908 deprecation.py:323] From c:\program files\python37\lib\site-packages\tensorflow_core\python\ops\math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0407 17:31:56.945195 12908 module_wrapper.py:139] From c:\program files\python37\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0407 17:31:56.952196 12908 mod

Train on 6400 samples, validate on 1601 samples
Epoch 1/500
6400/6400 [==============================] - 1s 138us/step - loss: 1.0203 - accuracy: 0.5311 - val_loss: 0.5293 - val_accuracy: 0.8001


W0407 17:31:58.326274 12908 module_wrapper.py:139] From c:\program files\python37\lib\site-packages\keras\callbacks\tensorboard_v1.py:343: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.



Epoch 2/500
6400/6400 [==============================] - 1s 93us/step - loss: 0.4947 - accuracy: 0.8175 - val_loss: 0.3097 - val_accuracy: 0.8969
Epoch 3/500
6400/6400 [==============================] - 1s 98us/step - loss: 0.3639 - accuracy: 0.8770 - val_loss: 0.2670 - val_accuracy: 0.9219
Epoch 4/500
6400/6400 [==============================] - 1s 101us/step - loss: 0.3096 - accuracy: 0.8981 - val_loss: 0.2583 - val_accuracy: 0.9207
Epoch 5/500
6400/6400 [==============================] - 1s 111us/step - loss: 0.2765 - accuracy: 0.9108 - val_loss: 0.2650 - val_accuracy: 0.9157
Epoch 6/500
6400/6400 [==============================] - 1s 90us/step - loss: 0.2583 - accuracy: 0.9172 - val_loss: 0.2295 - val_accuracy: 0.9294
Epoch 7/500
6400/6400 [==============================] - 1s 101us/step - loss: 0.2414 - accuracy: 0.9244 - val_loss: 0.2407 - val_accuracy: 0.9194
Epoch 8/500
6400/6400 [==============================] - 1s 106us/step - loss: 0.2308 - accuracy: 0.9264 - val_loss: 0.22

6400/6400 [==============================] - 1s 169us/step - loss: 0.2494 - accuracy: 0.9244 - val_loss: 0.2573 - val_accuracy: 0.9263
Epoch 18/500
6400/6400 [==============================] - 1s 171us/step - loss: 0.2459 - accuracy: 0.9286 - val_loss: 0.2295 - val_accuracy: 0.9294
Epoch 19/500
6400/6400 [==============================] - 1s 179us/step - loss: 0.2352 - accuracy: 0.9284 - val_loss: 0.2487 - val_accuracy: 0.9219
128-nodes-8-dense-1586260966
SCORE: [0.24092333730666146, 0.9189594984054565]


In [4]:
from sklearn.metrics import classification_report as cr
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

In [5]:
ob1=KNeighborsClassifier(3)
ob3=DecisionTreeClassifier()
ob4=RandomForestClassifier()
#ob5=AdaBoostClassifier()
#ob6=GradientBoostingClassifier()
#ob7=XGBClassifier()

algorithms = [('knc', ob1),('dtc', ob3),('rcf', ob4)]

classifiers = [
KNeighborsClassifier(3),
DecisionTreeClassifier(),
RandomForestClassifier(),
]

In [6]:
x=train_X
y=train_y
#test_X
#test_y
k_list = [2,3,4,5]
for classifier in classifiers:
    accu = []
    pipe = Pipeline(steps=[('classifier', classifier)])
    
    for j in k_list:
        accuracy = []
        kf = KFold(n_splits=j,shuffle=True)
        for (train, test), i in zip(kf.split(x, y), range(5)):
            pipe.fit(x[train], y[train])
            ypred=pipe.predict(x[test])
            accuracy.append(accuracy_score(y[test],ypred))
            #print(classifier)
            #print(cr(y[test],ypred))
        #print("\nList of Accuracies:",accuracy,"\n")
        acc = sum(accuracy)/j
        name=str(classifier).split('(')[0]
        print(name,"Average Accuracy:",acc,"\n")
        accu.append(acc)
    print("____________________________________________________________________________________________________________________________")

KNeighborsClassifier Average Accuracy: 0.5050617345663584 

KNeighborsClassifier Average Accuracy: 0.5243094613173354 

KNeighborsClassifier Average Accuracy: 0.5396814092953524 

KNeighborsClassifier Average Accuracy: 0.5446826202373517 

____________________________________________________________________________________________________________________________
DecisionTreeClassifier Average Accuracy: 0.8890146213446639 

DecisionTreeClassifier Average Accuracy: 0.8908886389201349 

DecisionTreeClassifier Average Accuracy: 0.908261431784108 

DecisionTreeClassifier Average Accuracy: 0.911884993753904 

____________________________________________________________________________________________________________________________
RandomForestClassifier Average Accuracy: 0.7504062734316421 

RandomForestClassifier Average Accuracy: 0.7751531058617673 

RandomForestClassifier Average Accuracy: 0.800401924037981 

RandomForestClassifier Average Accuracy: 0.8050244378513429 

_________________

In [7]:
print('Neural network scores:')
for i in scores:
    print('accuracy:',i[1])

Neural network scores:
accuracy: 0.9349674582481384
accuracy: 0.9299649596214294
accuracy: 0.9189594984054565
accuracy: 0.9189594984054565
